## ✅ Description
This is a STT which was made as an assignment for my Programming course from MSc Voice Technology at Rijksuniversiteit Groningen/Campus Fryslân.

The data is my 105 English recordings in [Common Voice](https://commonvoice.mozilla.org/en).

How I first did it was just like the notebook but then realized the link will be expired after some time so I just downloaded my data and uploaded it here so I  can train my STT easier.

## ✅ Upload My .txt file and .zip file

Just drag and drop the files in the /content and you are done.


### 👀 Take a look at your data

In [ ]:
# take a look at what you've uploaded
! ls .

## ✅ Install 🐸STT

In [ ]:
## Install Coqui STT
# dependencies
! apt-get install sox libsox-fmt-mp3 libopusfile0 libopus-dev libopusfile-dev
! pip install --upgrade pip
# the Coqui training package
! pip install coqui_stt_training
! pip uninstall -y tensorflow; pip install "tensorflow-gpu==1.15"
# code with importer scripts
! git clone --depth=1 https://github.com/coqui-ai/STT.git

## ✅ Import my data for 🐸STT



In [ ]:
! python STT/bin/import_cv_personal.py --normalize takeout_442_metadata.txt  takeout_442_pt_0.zip

# now we're going to split the dataset into {train,dev,test} with auto_import_dataset
from coqui_stt_training.util.config import initialize_globals_from_args
import glob

DATA_CSV=glob.glob("/content/takeout_442_pt_0/data.csv")[0]

initialize_globals_from_args(
    auto_input_dataset=DATA_CSV,
    save_checkpoint_dir='checkpoints/'
)

## ✅ Download a pre-trained English 🐸STT model

In [ ]:
### Download pre-trained model
import os
import tarfile
from coqui_stt_training.util.downloader import maybe_download

def download_pretrained_model():
    model_dir="english/"
    if not os.path.exists("english/coqui-stt-1.1.0-checkpoint"):
        maybe_download("model.tar.gz", model_dir, "https://github.com/coqui-ai/STT/releases/download/v1.1.0/coqui-stt-1.1.0-checkpoint.tar.gz")
        print('\nNo extracted pre-trained model found. Extracting now...')
        tar = tarfile.open("english/model.tar.gz")
        tar.extractall("english/")
        tar.close()
    else:
        print('Found pre-trained 🐸STT model, skipping download.')

def download_language_model():
    model_dir="english/"
    if not os.path.exists("english/huge-vocabulary.scorer"):
        maybe_download("huge-vocabulary.scorer", model_dir, "https://github.com/coqui-ai/STT-models/releases/download/english/coqui/v1.0.0-huge-vocab/huge-vocabulary.scorer")
    else:
        print('Found 🐸STT language model, skipping download.')

# Download + extract pre-trained English model
download_pretrained_model()
download_language_model()

## ✅ Customize to my Voice

Let's kick off a training run 🚀🚀🚀 and fine-tune the pre-trained 🐸STT model to my voice.

In [ ]:
from coqui_stt_training.util.config import initialize_globals_from_args
from coqui_stt_training.train import train
import glob

TRAIN_CSV = glob.glob("/content/takeout_442_pt_0/train.csv")
DEV_CSV = glob.glob("/content/takeout_442_pt_0/dev.csv")

initialize_globals_from_args(
    load_checkpoint_dir="english/coqui-stt-1.1.0-checkpoint",
    save_checkpoint_dir="my-model/checkpoints",
    alphabet_config_path="english/coqui-stt-1.1.0-checkpoint/alphabet.txt",
    train_files=TRAIN_CSV,
    dev_files=DEV_CSV,
    epochs=10,
    load_cudnn=True,
    train_batch_size=32,
    dev_batch_size=32,
)

train()

## ✅ Test the model

We made it, yeaaaaaaaaaay! 🙌

Let's kick off the testing run, and see how the model performs.

First, let's see how well the off-the-shelf English model recognizes my voice.

In [ ]:
# Test STT v1.0 on my voice data

from coqui_stt_training.util.config import initialize_globals_from_args
from coqui_stt_training.evaluate import test
import glob

TEST_CSV = glob.glob("/content/takeout_442_pt_0/test.csv")

initialize_globals_from_args(
    checkpoint_dir="/content/english/coqui-stt-1.1.0-checkpoint",
    scorer_path="english/huge-vocabulary.scorer",
    test_files = TEST_CSV,
    test_batch_size = 32,
)

test()

Now, let's see how much better ✨ my new custom model ✨ performs 🤩

In [ ]:
# Test my custom model on my held-out test data

from coqui_stt_training.util.config import initialize_globals_from_args
from coqui_stt_training.evaluate import test
import glob

TEST_CSV = glob.glob("/content/takeout_442_pt_0/test.csv")

initialize_globals_from_args(
    checkpoint_dir="my-model/checkpoints",
    scorer_path="english/huge-vocabulary.scorer",
    test_files = TEST_CSV,
    test_batch_size=32,
)

test()